In [ ]:
import os
import numpy as np
import pickle
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
%matplotlib inline

import nept

from loading_data import get_data
from utils_maze import find_zones, speed_threshold

thisdir = os.path.realpath(os.getcwd())
pickle_filepath = os.path.join(thisdir, 'cache', 'pickled')

from analyze_decode import get_likelihoods, get_decoded

In [ ]:
from run import spike_sorted_infos, info

session = info.r063d2

neurons_filename = session.session_id + '_neurons.pkl'
pickled_neurons = os.path.join(pickle_filepath, neurons_filename)
with open(pickled_neurons, 'rb') as fileobj:
    neurons = pickle.load(fileobj)
 
# For whole phase 3
t_start = session.task_times["phase3"].start
t_stop = session.task_times["phase3"].stop
# General bad region
#t_start, t_stop = (5580, 5630)
# Advance better than noadvance
#t_start, t_stop = (5585, 5595)
# Advance skips bad points
#t_start, t_stop = (5609, 5616)

dargs = dict(sequence_speed=5.,
             sequence_len=3,
             min_epochs=3)
largs = dict(speed_limit=0.4,
             min_swr=3,
             min_neurons=2,
             min_spikes=1,
             experiment_time="phase3",
             neurons=neurons,
             shuffle_id=False,
             info=session)

In [ ]:
def analyze(window_size, window_advance, gaussian_std):
    _largs = largs.copy()
    _largs["window_size"] = window_size
    _largs["window_advance"] = window_advance
    _largs["gaussian_std"] = gaussian_std
    d = {}
    
    gl = get_likelihoods(**_largs)
    d['likelihood'] = gl[0]
    # print(gl[1])  # time_edges
    d['decoded'], d['decoded_epochs'], d['errors'] = get_decoded(*gl, **dargs)
    
    print('window %.4f advancing in %.4f with %s std: %.3f' % (
        window_size, window_advance, gaussian_std, np.mean(d['errors'])))
    
    return d

In [ ]:
nooverlap = analyze(0.0125, 0.0125, 0.0075)
overlap = analyze(0.025, 0.0125, None)

In [ ]:
print(nooverlap["errors"].shape)
print(overlap["errors"].shape)

In [ ]:
def hist(d):
    plt.figure()
    plt.hist(d["errors"])

hist(nooverlap)
hist(overlap)

In [ ]:
def p(d, left, right):
    t = d['decoded'].time
    error = d['errors']
    errorsig = nept.AnalogSignal(error, t).time_slice(left, right)
    print("n_points: %d, mean error: %.3f" % (
        errorsig.n_samples, np.mean(errorsig.data)))
    plt.plot(t, error, ls="none", marker=".")
    plt.xlim(left, right)
    
def comp(d1, d2, left, right):
    plt.figure()
    p(d1, left, right)
    p(d2, left, right)

In [ ]:
comp(nooverlap, overlap, 5500, 8000)

In [ ]:
comp(nooverlap, overlap, 5580, 5630)

In [ ]:
comp(nooverlap, overlap, 5585, 5595)